In [1]:
import pandas as pd
import scipy.io
import os
import mne
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

from mne.channels import make_standard_montage

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


In [2]:
data_dir = os.path.dirname("./data/")
data_files = os.listdir(data_dir)

In [3]:
def annotations_from_eGUI(raw, egui):
    codes = []
    starts = []
    
    current_state = None
    
    for i in range(len(egui)):
        if egui[i][0] != current_state:
            starts.append(i)
            current_state = egui[i][0]
            codes.append(str(egui[i][0]))
    
    starts.append(len(egui))
    codes = np.array(codes)
    sf = raw.info.get('sfreq')
    starts = np.array(starts) / sf
    durations = starts[1:] - starts[:-1]
    starts = starts[:-1]
    
    raw.set_annotations(mne.Annotations(onset=starts, duration=durations, description=codes))
            

def raw_from_mat(file):
    mat = scipy.io.loadmat(os.path.join(data_dir, file))
    
    sampling_freq = mat["o"][0][0][2][0][0]
    n_samples = mat["o"][0][0][3][0][0]
    ch_names = [element[0][0] for element in mat["o"][0][0][6]]
    
    df = pd.DataFrame(mat["o"][0][0][5], columns=ch_names)
    df = df.drop(columns=["X5"])
    df = df.T
    ch_names.remove("X5")
    
    ch_types = ['eeg'] * 21
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
    raw = mne.io.RawArray(df.to_numpy(), info)
    
    montage = make_standard_montage("standard_prefixed")
    raw.set_montage(montage)
    
    raw.load_data().set_eeg_reference(ref_channels='average')
    annotations_from_eGUI(raw, mat["o"][0][0][4])
    return raw


def filter_raw(raw):
    return raw.load_data().filter(0.1, 30, method="fir", phase="zero-double")

In [4]:
raw_NoMT = [raw_from_mat(file) for file in data_files if "NoMT" in file]
raw_FREEFORM = [raw_from_mat(file) for file in data_files if "FREEFORM" in file]

Creating RawArray with float64 data, n_channels=21, n_times=664400
    Range : 0 ... 664399 =      0.000 ...  3321.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=664600
    Range : 0 ... 664599 =      0.000 ...  3322.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=662400
    Range : 0 ... 662399 =      0.000 ...  3311.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=667600
    Range : 0 ... 667599 =      0.000 ...  3337.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 d

In [5]:
def get_epochs(raw):
    metadata_tmin, metadata_tmax = -1, 1

    all_events, all_event_id = mne.events_from_annotations(raw)
    metadata, events, event_id = mne.epochs.make_metadata(
        events=all_events,
        event_id=all_event_id,
        tmin=metadata_tmin,
        tmax=metadata_tmax,
        sfreq=raw.info["sfreq"],
    )
    return mne.Epochs(raw, events, event_id)


In [6]:
epochs_NoMT = [get_epochs(file) for file in raw_NoMT]
epochs_FREEFORM = [get_epochs(file) for file in raw_FREEFORM]


Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1931 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1919 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1925 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1935 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used

In [7]:
epochs_NoMT[0].get_data().max()

Using data from preloaded Raw for 1931 events and 141 original time points ...
1 bad epochs dropped


865.3172938443672

In [8]:
epochs_FREEFORM[0].get_data().max()

Using data from preloaded Raw for 1481 events and 141 original time points ...
1 bad epochs dropped


158.92792102206735

In [9]:
epochs_data_NOMT = [file.get_data() for file in epochs_NoMT]
epochs_data_FREEFORM = [file.get_data() for file in epochs_FREEFORM]

Using data from preloaded Raw for 1930 events and 141 original time points ...
Using data from preloaded Raw for 1919 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1925 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1935 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1935 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1935 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1933 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1480 events and 141 original time points ...
Using data from preloaded Raw for 1383 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1409 events and 141 original time points ...
1 bad epochs dropped


In [10]:
stacked_NOMT = np.vstack(epochs_data_NOMT)
stacked_FREEFORM = np.vstack(epochs_data_FREEFORM)

In [11]:
df_No_MT = pd.DataFrame().append(raw_NoMT,ignore_index=True)
df_FREE = pd.DataFrame().append(raw_FREEFORM,ignore_index=True)


In [12]:
df_No_MT.shape

(7, 1)

In [13]:
df_FREE

ValueError: All picks must be < n_channels (21), got 21

ValueError: All picks must be < n_channels (21), got 21

In [14]:
def epoch_Data(arr,action):
    res=[]
    obj=[]
    for i in tqdm(arr):
        try:
            float(i[-1])
        except ValueError:
            if len(obj)>0:
                 res.append(obj)
            obj=[] 
        else:
       
            if int(i[-1])==action:
                obj.append(i)
            else:
                if len(obj)>0:
                    res.append(obj)
                obj=[] 
    return res
        
        

In [15]:
def make_all_same_length(data,length):
      return [i[0:length] for i in data]
        

In [16]:
def epoch_Data_2(arr,action):
    res=[]
    obj=[]
    for ind,i in tqdm(enumerate(arr)):
        try:
            float(i[-1])
        except ValueError:
            if len(obj)>0:
                 res.append(obj)
            obj=[] 
        else:
                if int(i[-1])==action:
                    if len(obj)==0:
                        obj.extend([*arr[ind-200:ind+1]])
                    else:
                        obj.append(i)
                else:
                    if len(obj)>0:
                        res.append(obj)
                    obj=[] 
    return res
        
        

In [17]:
arr_NoMT=df_No_MT.to_numpy()
arr_FREE=df_FREE.to_numpy()


In [18]:
splitted_nomt=epoch_Data(arr_NoMT,0)
splitted_free=epoch_Data_2(arr_FREE,1)

  0%|          | 0/7 [00:00<?, ?it/s]


TypeError: float() argument must be a string or a number, not 'RawArray'

In [ ]:
splitted_and_filtered_nomt=[i for i in splitted_nomt if len(i)<500 and len(i)>400]
splitted_and_filtered_free=[i for i in splitted_free if len(i)<500 and len(i)>400]
len(splitted_and_filtered_free)
len(splitted_and_filtered_nomt)

In [ ]:
splitted_filtered_cropped_nomt=make_all_same_length(splitted_and_filtered_nomt,400)
splitted_filtered_cropped_free=make_all_same_length(splitted_and_filtered_free,400)


In [ ]:
X_nomt=np.array(splitted_filtered_cropped_nomt)
X_nomt=np.swapaxes(X_nomt,1,2)
X_free=np.array(splitted_filtered_cropped_free)
X_free=np.swapaxes(X_free,1,2)

In [ ]:
X_nomt.shape

In [ ]:
X_free.shape

In [ ]:
X_nomt_train=X_nomt[:3000]
X_nomt_test=X_nomt[3000:]

In [19]:
# Overwriting
X_nomt_train=stacked_NOMT[:12000]
X_nomt_test=stacked_NOMT[12000:]

In [33]:
X_free = stacked_FREEFORM

In [30]:
stacked_NOMT.shape

(13506, 21, 141)

In [31]:
stacked_FREEFORM.shape

(4270, 21, 141)

In [20]:
X_free_test = stacked_FREEFORM

In [36]:
import random
idy=random.sample(range(0, len(X_free)), X_nomt_test.shape[0])
X_free_test=X_free[idy]

In [37]:
X_free_test.shape


(1506, 21, 141)

In [38]:
from tensorflow.keras import layers
layer = layers.Normalization()
layer1 = layers.Normalization()
layer.adapt(X_nomt_train.astype(float))
layer1.adapt(X_free_test.astype(float))

print(X_nomt_train)
print(np.max(X_nomt_train))
print(np.max(X_free_test))
print(np.max(layer(X_nomt_train)))
print(np.max(layer1(X_free_test)))

[[[-1.63645528e+01 -1.78597909e+01 -1.72355052e+01 ...  2.05992567e+01
    1.57678281e+01  2.30483043e+01]
  [-1.50679675e+01 -1.48632056e+01 -1.32889199e+01 ...  2.25658420e+01
    2.76144135e+01  2.28048897e+01]
  [-5.44406504e+00 -4.81930314e+00 -5.08501742e+00 ...  1.26597445e+01
    1.30783159e+01  1.35087921e+01]
  ...
  [-6.03357724e+00 -6.85881533e+00 -8.78452962e+00 ...  1.76402323e+01
    1.80188037e+01  1.52792799e+01]
  [ 9.74300813e+00  9.17777003e+00  8.53205575e+00 ...  5.69681765e+00
    5.51538908e+00  4.91586527e+00]
  [ 4.21959350e+00  5.12435540e+00  5.01864111e+00 ... -2.71165970e+01
   -2.69080256e+01 -2.90475494e+01]]

 [[-4.44094077e+00 -5.20570267e+00 -1.29855981e+00 ...  1.06953542e+00
    4.77620209e+00  1.02604878e+01]
  [ 7.19905923e+00 -2.94570267e+00 -8.08559814e-01 ...  1.17695354e+01
    1.31062021e+01  1.03904878e+01]
  [-1.75825784e+00 -6.23019744e-01 -3.85587689e+00 ... -3.24778165e+00
   -3.94111498e+00 -2.67682927e+00]
  ...
  [-2.43362369e+00 -6.0

In [45]:
latent_dim = 512 
keras.backend.clear_session()
class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(1024, activation='gelu'),
      layers.Dense(512, activation='gelu'),
      layers.Dense(64, activation='gelu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(512, activation='gelu'),
      layers.Dense(1024, activation='gelu'),
      layers.Dense(21*141, activation='linear'),
      layers.Reshape((21, 141))
    ])
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)

In [46]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError())
autoencoder.fit(X_nomt_train, X_nomt_train,
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_data=(X_nomt_test[:200], X_nomt_test[:200]))

Epoch 1/50
188/188 [==============================] - 6s 28ms/step - loss: 122.6125 - val_loss: 19.4592
Epoch 2/50
188/188 [==============================] - 5s 27ms/step - loss: 65.4219 - val_loss: 17.9452
Epoch 3/50
188/188 [==============================] - 5s 28ms/step - loss: 56.9046 - val_loss: 16.5008
Epoch 4/50
 57/188 [========>.....................] - ETA: 3s - loss: 54.1314

KeyboardInterrupt: 

In [41]:
X_free_test.shape


(1506, 21, 141)

In [48]:
err=[]
err2=[]
for i in X_nomt_train:
    encoded = autoencoder.encoder(i).numpy()
    print(encoded.shape)
    decoded = autoencoder.decoder(encoded).numpy()
    print(decoded.shape)
    #print("Error:",(np.square(i-decoded)).mean())
    err.append((np.square(i-decoded)).mean())
print("###################")


for j in X_free_test:
    encoded = autoencoder.encoder(j).numpy()
    decoded = autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(layer(j)-decoded)).mean())
    err2.append((np.square(j-decoded)).mean())
print((np.square(autoencoder.call(X_free_test)-X_free_test)).mean())
print("##############")
print(np.array(err).mean())
print(np.array(err2).mean())


ValueError: Exception encountered when calling layer 'sequential' (type Sequential).

Input 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 2961, but received input with shape (21, 141)

Call arguments received by layer 'sequential' (type Sequential):
  • inputs=tf.Tensor(shape=(21, 141), dtype=float64)
  • training=None
  • mask=None

descicion 0.65

In [ ]:
def acc_Calc(a,b,th):
    first=[1 if i<th else 0 for i in a]
    last=[1 if i>th else 0 for i in b]
    return sum(first+last)/len(first+last)

In [ ]:
acc_Calc(err,err2,0.2)

In [ ]:
print(err[:10])

1,1,1,1,1,1,,0,1,1,1 9/10

In [ ]:
print(err2[:10])

0,0,1,0,1,1,1,1,0,0 5/10

Acc:

In [ ]:
14/20